# Lab 7/tAPP 9 continued: Column-Wise Pasting and Merging of Flat-File Databases with Bash 

**Author:** 

**Team:**  

**Team Members Present:** 

**Date:** Mar 12, 2025

**Course:** DSC 011 Sp25

## The Goal and Task of This Assignment

**The Goal of this assignment is to provide a workbook for you to practice pasting data from multiple files together column-wise.** 


**Your Task:  follow the instructor's demonstration and copy, develop and/or debug codes in this notebook and execute them in the code cells as instructed so their output appears in the notebook, with the aim of reproducing the calculations so that the notebook evaluates the code is CORRECT. If the solution is INCORRECT, you must ask your teammates or the instructor for help fixing the bug or you will lose credit. At the end of lecture, export your notebook to HTML and upload to CatCourses.**

## General Instructions:
1. In this notebook file, please fill in all information at the top and save the notebook.
2. Rename your saved notebook using your own name.
3. Work the assignment as detailed below. 
4. When finished working the assignment, save the notebook, then export it to HTML.
5. Upload the HTML export of your worked assignment to CatCourses.

## Specific Instructions: 


## Validate Existence and Correctness of Baby Names Input File

1. **Run the following code cell with shift-enter to validate the existence and correctness of the baby names input data.**

In [ ]:
# perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt | cut -c 6-10,15- | tr -d " ," | tr ":|" , > baby_clean.csv
if [ `cat baby_clean.csv | wc -l` -eq 2800 ]; then 
    echo "2800 data lines detected in baby_clean.csv. CORRECT."
else 
    echo "Fewer or Greater than 2800 data lines detected in baby_clean.csv. INCORRECT. Amoungt found was:"
    cat baby_clean.csv | wc -l
    echo "Investigate the output interactively in the cell below using wc, egrep, sort, cut, etc"
fi;
echo "testing equality of data..."
echo "...first ten lines..."
head baby_clean.csv
echo "...md5sum..."
head baby_clean.csv | md5sum
if [[ `head baby_clean.csv | md5sum` == "6d8913fae285edec358d107db11dc4ed  -" ]]; then 
    echo "md5sum of first ten lines of baby_clean.csv is CORRECT."
else 
    echo "ERROR: md5sum of first ten lines of baby_clean.csv is INCORRECT."
fi;
echo "...all data..."
echo "...md5sum..."
cat baby_clean.csv | md5sum
if [[ `cat baby_clean.csv | md5sum` == "8fee16f6cd371c349dc6f30c04e4648c  -" ]]; then 
    echo "md5sum of all data in baby_clean.csv is CORRECT."
else 
    echo "ERROR: md5sum of all data in baby_clean.csv INCORRECT."
fi;

## Compute Observation-Level and Population-Level Frequency Statistics for First Letters of Baby Names for Boys, Girls, and Both Sexes and Save Them to File

We are working towards making a processed data-file to compare baby-name first-letter frequencies **pooled over** 14 decades in six different ways: stratified by sex or pooled over sex (3 possibilities), and at the "observation level" or "population-level" (2 possibilities for a total of $3 \times 2 = 6$ ways). 

Therefore we wish to make a single file flat-file database of first-letter frequencies containing six rows, one for each letter of the alphabet, and seven columns, with letters in teh first column and the other six columns showing **absolute frequencies** (also known as **counts data**) of first-letters over the 14 decades among boys, girls, or both at both population and individual levels of counting. 

In statistics, for this data:

1. the alphabet letters,
2. the sex of babies (boy, girl or pooled) and
3. the way of counting letters (observation or population levels)

are all **explanatory variables** and the frequencies or counts that we have computed for each combination of explanatory variables are **response variables**. Explanatory variables represent controlled or validated inputs to an experimental or observational study and subsequent data science analysis. Explanatory variables help explain response variables, which are like outputs of such studies. In this study on letters, the data has a **frequency response.** In this case, the letters also represent inputs to the associative array we used to compute them as keys whose values are the frequency responses we tallied for them.

We calculated and validated our letter frequencies using Bash+Perl one-liners, using separate code for each of the six combinations of explanatory variables (boys/girls/pooled vs individual/population). We saved each calculation to a separate file. 

**Today we will learn how to use Bash to merge files column-wise, so as to make a flatfile database from separate files of data that have been cleaned and sorted so that all of the rows correspond to the letters in the same order.**

### Generate the Six Frequency Response Files for Different Explanatory Variables 

Previously we sorted our calculations for baby names first letters by the frequency response to make it easier to read the rank trends in popularity of letters. But in order to make our database master file, we need to make each corresponding row of our six source input files comparable, that is corresponding to the same first letter. The easiest way to do this is to sort alphabetically by the letters instead of by the frequency responses.  

2. **Modify the following code to generate the two intermediate frequency calculations for boys so that they are:**
   1. **sorted by letter instead of by frequency response.**
   2. **have no leading white space on each line so each line starts with frequencies (Hint: use `tr -s` and `cut -c`), and**
   3. **the two fields are separated by the tab character `\t`. (Hint: the alignment of letters will look a little funny when the output is tab-delimited.)**
**Validate your solution and ask for help as needed until you get all CORRECT at the bottom output.**

In [ ]:
# Compute observation-level frequency distribution of first letters of boys' names

echo "Computing Frequency Distribution of First Letters Among Top-Ranked Boys' Names Over 14 Decades Sorted by Letter:"
cut -d, -f3 baby_clean.csv | tr -d a-z | sort | uniq -c | sort -nr > boys_letters_obs.txt 
echo "Saved to file \"boys_letters_obs.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' boys_letters_obs.txt)
output=$(cat boys_letters_obs.txt)
md5sum=$(cat boys_letters_obs.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "c4bd3acc478a890e86f62bda79adb2a7" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 2800 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

echo ""

# Compute population-level frequencies of first letters
echo "Computing Birth-Weighted Frequency Distribution of First Letters Among Top-Ranked Boys' Names Over 14 Decades:"
cut -d, -f3,4 baby_clean.csv | tr -d a-z | 
perl -F, -ane '$f{$F[0]} += $F[1];END{foreach my $k (sort keys %f) { print "$f{$k} $k\n"}}' | 
sort -nr > boys_letters_pop.txt
echo "Saved to file \"boys_letters_pop.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' boys_letters_pop.txt)
output=$(cat boys_letters_pop.txt)
md5sum=$(cat boys_letters_pop.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "17709f34a15cb560dd53cdeab5d92f03" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 139453978 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

3. **Modify the following code to generate the two intermediate frequency calculations for girls so that they are:**
   1. **sorted by letter instead of by frequency response.**
   2. **have no leading white space on each line so each line starts with frequencies (Hint: use `tr -s` and `cut -c`), and**
   3. **the two fields are separated by the tab character `\t`. (Hint: the alignment of letters will look a little funny when the output is tab-delimited.)**
**Validate your solution and ask for help as needed until you get all CORRECT at the bottom output.**

In [ ]:
# Compute observation-level frequency distribution of first letters of girls' names
echo "Computing Frequency Distribution of First Letters Among Top-Ranked Girls' Names Over 14 Decades:"
cut -d, -f5 baby_clean.csv | tr -d a-z | sort | uniq -c | sort -nr > girls_letters_obs.txt
echo "Saved to file \"girls_letters_obs.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' girls_letters_obs.txt)
output=$(cat girls_letters_obs.txt)
md5sum=$(cat girls_letters_obs.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "8b38336146a98cfbff17e975aff36bbf" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 2800 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

echo ""

# Compute population-level frequencies of first letters
echo "Computing Birth-Weighted Frequency Distribution of First Letters Among Top-Ranked Girls' Names Over 14 Decades:"
cut -d, -f5,6 baby_clean.csv | tr -d a-z | 
perl -F, -ane '$f{$F[0]} += $F[1];END{foreach my $k (sort keys %f) { print "$f{$k} $k\n"}}' |
sort -nr  > girls_letters_pop.txt
echo "Saved to file \"girls_letters_pop.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' girls_letters_pop.txt)
output=$(cat girls_letters_pop.txt)
md5sum=$(cat girls_letters_pop.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "c14e61f4a54b23e2997bd39c360ac4be" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 119369271 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

3. **Modify the following code to generate the two intermediate frequency calculations pooling both girls and boys so that they are:**
   1. **sorted by letter instead of by frequency response.**
   2. **have no leading white space on each line so each line starts with frequencies (Hint: use `tr -s` and `cut -c`), and**
   3. **the two fields are separated by the tab character `\t`. (Hint: the alignment of letters will look a little funny when the output is tab-delimited.)**
**Validate your solution and ask for help as needed until you get all CORRECT at the bottom output.**

In [ ]:
# Compute observation-level frequency distribution of first letters of boys' and girls' names combined
echo "Computing Frequency Distribution of First Letters Among Top-Ranked Names Over Both Sexes and 14 Decades:"
cut -d, -f3,5 baby_clean.csv | tr , '\n' | tr -d a-z | sort | uniq -c | sort -nr > both_letters_obs.txt
echo "Saved to file \"both_letters_obs.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' both_letters_obs.txt)
output=$(cat both_letters_obs.txt)
md5sum=$(cat both_letters_obs.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "af6660e25abe8b084557d0fac3e8c483" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 5600 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

echo ""

# Compute population-level frequencies of first letters
echo "Computing Birth-Weighted Frequency Distribution of First Letters Among Top-Ranked Names Over Both Sexes and 14 Decades:"
cut -d, -f3,4,5,6 baby_clean.csv | tr -d a-z | 
perl -F, -ane '$f{$F[0]} += $F[1];$f{$F[2]} += $F[3];END{foreach my $k (sort keys %f) { print "$f{$k} $k\n"}}' |
sort -nr  > both_letters_pop.txt
echo "Saved to file \"both_letters_pop.txt\""

# Validate output
echo "Validating..."
sum_of_freqs=$(perl -ane '$sum+=$F[0];END{print "$sum\n"}' both_letters_pop.txt)
output=$(cat both_letters_pop.txt)
md5sum=$(cat both_letters_pop.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "c4bcc2e596a7ad84757bb718a2302b45" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;
echo "## The sum of first-letter frequencies was:"
echo $sum_of_freqs
if [[ $sum_of_freqs -eq 258823249 ]]; then 
    echo "#### sum of frequencies of output is CORRECT."
else 
    echo "#### ERROR: sum of frequencies of the output is INCORRECT."
fi;

### Think About the Data Obtained So Far and Check Intermediate Work 

**Answer two Factual Questions and Make a Hypothesis. **

5. **Answer this Factual Question by inspecting your calculations in this notebook above: Are there more, fewer or approximately equal numbers of boy and girl babies represented in this data? Click into this Markdown Cell and write a short individual response. YOUR ANSWER:**

6. **Answer this Factual Question by using the code cell below to compute the answer to the following question with Python, Perl or Bash: what is the proportion of boy babies in the data?**


7. **Make a Hypothesis. Think about what could be various causes or explanations of your results obtained in this section so far. Click into this Markdown Cell and write a short individual description of your foremost explanations for this result. Please write for yourself before sharing and comparing your answers across your team.:**

8. **Inspect your Intermediate Data Files:** Click on the six files created by this notebook and verify that they look correct, with letters sorted alphabetically.

### Write and validate a Shell Glob to Match only the Six Intermediate Files

It's possible that you may have other intermediate processed files on your filesystem that we would wish to exclude from merging by column. We need to make write a shell glob that matches only exactly the six files that we want to merge. The six files have the names: 
    1. `boys_letters_pop.txt`
    2. `boys_letters_obs.txt`
    3. `girls_letters_pop.txt`
    4. `girls_letters_obs.txt`
    5. `both_letters_pop.txt`
    6. `both_letters_obs.txt`

9. **Provide a shell glob argument to the following command that matches only and exactly these six filenames and take note of the order of files.**

### Check How Many Letters/Lines/Rows Appear in Each File

10. **Use your shell glob to write a single bash one-liner command to check whether all intermediate input files contain the same numbers of lines and to verify that the population and observation level files, which they must. Click into this markdown file to briefly describe your findings. ANSWER:**

### Paste the Pooled Boys Data Together and Validate the Corresponding Rows

11. If all the input files contained the same letters in corresponding rows, we could use the bash command `paste` to merge files column-wise. This data is more complicated, because there is some missing data (some first letters of girls are not first letters of boys).

But the alternative more complicated command `join`, which can handle missing explanatory variables when merging rows, can only merge two input files. So we will make our master-input file in two steps. First we will make two more intermediate files each merging the files with the same number of lines, one for just the two files for boys, and the other for the other four files. We'll inspect the two files and make sure they all are merging corresponding rows for the same letters.

**Write a Bash command `paste` in the next code cell to verify that the two boys data files report data for the same letters in the same order and that the other four data files do the same:**
The output should have four columns and the beginning should look like:

```
8901618	A	224	A
6157982	B	146	B
10212882	C	226	C
```

### Paste the Pooled Boys Data Together and Subset and Try to Reorder the Columns with Cut

It is worth noting that `cut -f` allows you to select fields, but it cannot reorder them. By the way, tab characters are the default delimiter for `cut`. 

12. **Extend the previous Bash one-liner using `paste` and try to select and reorder the field with `cut -f` trying to output these columns in the order**

   1. letters
   2. boys -- population level
   3. boys -- observation level

The output should look like:
```
8901618	A	224
6157982	B	146
10212882	C	226
```

### Paste the Pooled Boys Data Together and Subset and Reorder the Columns with Perl

12. **Replace `cut` in your previous solution one-liner with the following Perl one-liner, modified by specifying which fields (rememer, `perl` is 0-indexed, unlike `cut`) that you want to print inside the square brackets (explanation, this uses what's called a "Perl array slice"). Select the following columns. After you validate the output, save it to the file `boys_paste.txt`:**

   1. letters
   2. boys -- population level
   3. boys -- observation level

The output should look like:
```
A	8901618	224
B	6157982	146
C	10212882	226
```

In [ ]:
paste boys_letters_pop.txt boys_letters_obs.txt | 
perl -ane 'print join "\t",@F[];print "\n"' > boys_paste.txt

# Validate output
echo "Validating..."
output=$(cat boys_paste.txt)
md5sum=$(cat boys_paste.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "89da09bdff438d3267ac583df17a5622" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;

### Paste the Pooled Other Data Together and Subset and Reorder the Columns with Perl

12. **Modify the following Perl one-liner by specifying which fields that you want to print inside the square brackets (explanation, this uses what's called a "Perl array slice"). Select the following columns. After you validate the output, save it to the file `other_paste.txt`:**

   1. letters
   2. girls -- population level
   3. girls -- observation level
   4. boys+girls ("both") -- population level
   5. boys+girls ("both") -- observation level

The output should look like:
```
A	11547024	272	20448642	496
B	6401778	136	12559760	282
C	8364906	203	18577788	429
D	6912345	120	18132130	293
```

In [ ]:
paste girls_letters_pop.txt girls_letters_obs.txt both_letters_pop.txt both_letters_obs.txt | 
perl -ane 'print join "\t",@F[];print "\n"' > other_paste.txt

# Validate output
echo "Validating..."
output=$(cat other_paste.txt)
md5sum=$(cat other_paste.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "a3b4a3f71cb6ef67ca40cdde72d3d99a" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;

### Join the Boys Data With the Other Data Merging on Letters

12. **Run the first code chunk to check the documentation for the `join` command. In the next code cell, modify the bash one-liner by using the `-j` option to specify the join field (the letters) of both files, and the `-a` option to specify outputing all rows in the `other` file that don't match rows in the `boy` file. After checking your output, use shell redirection to create the master merged data file `baby_letters.txt` with 25 rows and 7 columns in the order:**

       1. letters
       2. boys -- population level
       3. boys -- observation level
       4. girls -- population level
       5. girls -- observation level
       6. boys+girls ("both") -- population level
       7. boys+girls ("both") -- observation level

The output should look like:
```
A 8901618 224 11547024 272 20448642 496
B 6157982 146 6401778 136 12559760 282
C 10212882 226 8364906 203 18577788 429
D 11219785 173 6912345 120 18132130 293
```

In [ ]:
join -e '0' -o '0,1.2,1.3,2.2,2.3,2.4,2.5' boys_paste.txt other_paste.txt > baby_letters.txt
# Validate output
echo "Validating..."
output=$(cat baby_letters.txt)
md5sum=$(cat baby_letters.txt| md5sum | perl -ane 'print "$F[0]\n"')
# Echo the validations
echo "## The output was:"
echo "$output"
echo "## The md5sum of the output file was:"
echo $md5sum
if [[ $md5sum == "5ae6736dd187d7cbadf61bc5de42b89b" ]]; then 
    echo "#### md5sum of the output is CORRECT."
else 
    echo "#### ERROR: md5sum of the output is INCORRECT."
fi;

**Please export your worked lab notebook containing command output to HTML and upload it to CatCourses at the end of Lab.**